# Filter methods

## Setup

In [58]:
import numpy as np
import pandas as pd
import altair as alt

from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

## Data

### Import data

In [59]:
df = pd.read_csv('https://raw.githubusercontent.com/kirenz/datasets/master/advertising.csv')

### Data structure

In [60]:
df

,Market,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Market     200 non-null    int64  
 1   TV         200 non-null    float64
 2   radio      200 non-null    float64
 3   newspaper  200 non-null    float64
 4   sales      200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


### Data corrections

In [62]:
# variable Market is categorical
df['Market'] = df['Market'].astype('category')

### Variable lists

In [63]:
# define outcome variable as y_label
y_label = 'sales'

# select features
features = df.drop(columns=[y_label, 'Market']).columns

# create feature data
X = df[features]

# create response
y = df[y_label]

In [64]:
features

Index(['TV', 'radio', 'newspaper'], dtype='object')

### Data splitting

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)

In [66]:
# data exploration set
df_train = pd.DataFrame(X_train.copy())
df_train = df_train.join(pd.DataFrame(y_train))

### Analyze data

In [67]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
TV,160.0,150.019375,84.418857,0.7,77.750,150.65,218.825,296.4
radio,160.0,22.875625,14.805216,0.0,9.825,21.20,36.425,49.6
newspaper,160.0,29.945625,20.336449,0.3,12.875,25.60,44.500,100.9
sales,160.0,14.100000,5.108754,1.6,10.475,13.20,17.325,27.0


In [68]:
alt.Chart(df_train).mark_bar().encode(
    alt.X(alt.repeat("column"), type="quantitative", bin=True),
    y='count()',
).properties(
    width=150,
    height=150
).repeat(
    column=['sales', 'TV', 'radio', 'newspaper']
)

alt.RepeatChart(...)

In [69]:
alt.Chart(df_train).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative')
).properties(
    width=150,
    height=150
).repeat(
    row=['sales', 'TV', 'radio', 'newspaper'],
    column=['sales', 'TV', 'radio', 'newspaper']
).interactive()

alt.RepeatChart(...)

In [70]:
# inspect correlation between outcome and possible predictors
corr = df_train.corr()
corr[y_label].sort_values(ascending=False)

sales        1.000000
TV           0.768874
radio        0.592373
newspaper    0.237874
Name: sales, dtype: float64

In [71]:
# take a look at all correlations
corr.style.background_gradient(cmap='Blues')

,TV,radio,newspaper,sales
TV,1.000000,0.053872,0.019084,0.768874
radio,0.053872,1.000000,0.388074,0.592373
newspaper,0.019084,0.388074,1.000000,0.237874
sales,0.768874,0.592373,0.237874,1.000000


## Model

### Select model

In [72]:
# select the linear regression model
reg = LinearRegression()

### Training & validation

In [73]:
# cross-validation with 5 folds
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error') *-1

In [74]:
# store cross-validation scores
df_scores = pd.DataFrame({"lr": scores})

# reset index to match the number of folds
df_scores.index += 1

# print dataframe
df_scores.style.background_gradient(cmap='Blues')

,lr
1,4.192954
2,1.500644
3,2.109080
4,2.541355
5,4.372931


In [75]:
alt.Chart(df_scores.reset_index()).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X("index", bin=False, title="Fold", axis=alt.Axis(tickCount=5)),
    y=alt.Y("lr", aggregate="mean", title="Mean squared error (MSE)")
)

alt.Chart(...)

In [76]:
df_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
lr,5.0,2.943393,1.279083,1.500644,2.10908,2.541355,4.192954,4.372931


### Fit model

In [77]:
# Fit the model to the complete training data
reg.fit(X_train, y_train)

LinearRegression()

### Coefficients

In [78]:
# intercept
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[reg.intercept_]}
    )

# make a slope table
slope = pd.DataFrame({
    "Name": features,
    "Coefficient": reg.coef_}
)

# combine estimates of intercept and slopes
table = pd.concat([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

,Name,Coefficient
0,Intercept,2.979
1,TV,0.045
2,radio,0.189
3,newspaper,0.003


### Evaluation on test set

In [79]:
# obtain predictions
y_pred = reg.predict(X_test)

In [80]:
# R squared
r2_score(y_test, y_pred).round(3)

0.899

In [81]:
# MSE
mean_squared_error(y_test, y_pred).round(3)

3.174

In [82]:
# RMSE
mean_squared_error(y_test, y_pred, squared=False).round(3)

1.782

In [83]:
# MAE
mean_absolute_error(y_test, y_pred).round(3)

1.461

### Feature importance

In [84]:
importance = np.abs(reg.coef_)

df_imp = pd.DataFrame({"coeff": importance, 
                       "name": features})
df_imp

,coeff,name
0,0.044730,TV
1,0.189195,radio
2,0.002761,newspaper


In [85]:
alt.Chart(df_imp).mark_bar().encode(
    x="coeff",
    y=alt.Y("name", sort='-x')
)

alt.Chart(...)

Now we want to select the two features which are the most important according to the coefficients. 

The SelectFromModel is meant just for that. 

SelectFromModel accepts a threshold parameter and will select the features whose importance (defined by the coefficients) are above this threshold.

Since we want to select only 2 features, we will set this threshold slightly above the coefficient of third most important feature.

In [96]:
# set threshold
threshold = np.sort(importance)[0] + 0.01 


# fit model
sfm = SelectFromModel(reg, threshold=threshold).fit(X_train, y_train)

In [99]:
features[sfm.get_support()]

Index(['TV', 'radio'], dtype='object')

In [100]:
print(f"Features selected by SelectFromModel: {features[sfm.get_support()]}")

Features selected by SelectFromModel: Index(['TV', 'radio'], dtype='object')


### Permutation feature importance

 Features that are deemed of low importance for a bad model (low cross-validation score) could be very important for a good model. Therefore it is always important to evaluate the predictive power of a model using a held-out set (or better with cross-validation) prior to computing importances. Permutation importance does not reflect to the intrinsic predictive value of a feature by itself but how important this feature is for a particular model.

Using a held-out set makes it possible to highlight which features contribute the most to the generalization power of the inspected model.

In [91]:
scoring = ['r2', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error']

In [92]:
r_multi = permutation_importance(reg, X_test, y_test,
                           n_repeats=30,
                           random_state=0,
                           scoring=scoring)

In [93]:
features

Index(['TV', 'radio', 'newspaper'], dtype='object')

In [94]:
for metric in r_multi:
    print(f"{metric}")
    r = r_multi[metric]
    for i in r.importances_mean.argsort()[::-1]:
      #  if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            print(f"    {features[i]:<8}"
                  f"{r.importances_mean[i]:.3f}"
                  f" +/- {r.importances_std[i]:.3f}")

r2
    TV      1.147 +/- 0.160
    radio   0.502 +/- 0.091
    newspaper-0.001 +/- 0.001
neg_mean_absolute_percentage_error
    TV      0.323 +/- 0.040
    radio   0.139 +/- 0.023
    newspaper-0.001 +/- 0.001
neg_mean_squared_error
    TV      36.191 +/- 5.042
    radio   15.832 +/- 2.883
    newspaper-0.035 +/- 0.040
